This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [5]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'szbecABBRF5T3hn9AEyz'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [6]:
# First, import the relevant modules
import requests
from collections import defaultdict
from itertools import tee

In [7]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
request_params = {'api_key': API_KEY, 'start_date': '2017-12-01', 'end_date': '2017-12-02'}
r = requests.get('https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json', params = request_params)

In [8]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume'], 'start_date': '2017-12-01', 'end_date': '2017-12-02', 'frequency': 'daily', 'data': [['2017-12-01', 176.03, 177.48, 172.34, 175.1, 19824531.0, 0.0, 1.0, 176.03, 177.48, 172.34, 175.1, 19824531.0]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Problem 1 and 2: collect data for AFX_X for 2017 and convert it to a python dictionary

I make a request to get the data from the quandl API, then use .json() to convert it to a dict.  I noticed that this dict has only one key associated with a dict value, so I use the inner dict as my raw data.

In [9]:
request_params = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31', 'ticker': 'AFX_X'}
r = requests.get('https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json', params=request_params)
raw_data = r.json()['dataset_data']
#print(raw_data)

Next I clean this data a little by making a new dictionary using the ```raw_data['column names']``` as keys mapping to a list of the corresponding values in ```raw_data['data']```

In [10]:
AFX_data = defaultdict(list)
for entry in raw_data['data']:
    for colname,item in zip(raw_data['column_names'],entry):
        AFX_data[colname].append(item)
print(AFX_data['High'][:5])

[178.85, 178.9367, 178.44, 177.0, 177.53]


# Problem 3: Calculate the highest and lowest opening prices in this period

In [11]:
# Use python's max and min functions on the appropriate column
high_open = max(AFX_data['Open'])
low_open = min(AFX_data['Open'])
print('Highest opening price: $' + str(high_open))
print('Lowest opening price: $' + str(low_open))

Highest opening price: $183.51
Lowest opening price: $116.03


# Problem 4: What was the largest change in any one day (based on High and Low price)?

In [12]:
# Create a list of (High - Low) for each day, then find the max
big_change = max([high-low for high, low in zip(AFX_data['High'], AFX_data['Low'])])
print('$' + str(big_change))

$8.97999999999999


# Problem 5: What was the largest change between any two days (based on Closing Price)?

In [18]:
# From itertools recipes
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [14]:
# zip() two lists of closing prices that are one day apart and find the max of a list of their differences
# This solution only gives the absolute value of the change, so it does not say if it was an increase or
# decrease
big_closing_change = max([abs(day1 - day2) for day1, day2 in pairwise(AFX_data['Close'])])
print('$' + str(big_closing_change))

$7.6699999999999875


# Problem 6: What was the average daily trading volume during this year?

In [15]:
# Use builtin sum() and len() functions on the 'Volume' column
print(sum(AFX_data['Volume']) / len(AFX_data['Volume']))

16468073.856


# Problem 7: What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [16]:
def median(data):
    """Calculate the median of data"""
    # Sort the data and return the middle value, or the average of the two middle values
    N = len(data)
    data_srt = sorted(data)
    if N % 2 != 0:
        return data_srt[N//2]
    else:
        return (data_srt[N//2] + data_srt[N//2 - 1]) / 2

In [17]:
print(median(AFX_data['Volume']))

14548954.0
